# TAREA SCRAPING

### Vamos a scrapear la lista de 100 libros mas vendidos de Cuspide

---

Arranco scrapeando los enlaces a cada uno de los 100 libros

In [1]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.cuspide.com/top100.aspx'
cuspide = requests.get(url)
soup = BeautifulSoup(cuspide.text, 'lxml')

libros = soup.find_all('article')

links = [libro.a.get('href') for libro in libros]

---

Voy a hacerle un request a cada enlace, para scrapear **titulo** y **precio** para luego guardar una lista de libros con url, titulo y precio

---

In [2]:
resultado = []

for link in links:
        url_libro = 'https://www.cuspide.com' + link
        try:
            dom = requests.get(url_libro)
            s = BeautifulSoup(dom.text, 'lxml')
        except:
            url_libro = f'Error en request a: {url_libro}'
        try:
            titulo = s.find('li', attrs={'class':'active'}).a.getText()
        except:
            titulo = 'Error al scrapear'
        try:
            precio = s.find('meta', attrs={'itemprop':'price'}).get('content').split('.')
            precio_ars = int(precio[0][1:] + precio[1][0:3])
        except:
            precio = '0'
        
        resultado.append([url_libro, titulo, precio_ars])

---

Una vez que tengo esa lista, abro conexion a la BD MySQL y creo una tabla para guardar la data scrpeada

---

In [3]:
#CONEXION ABIERTA
import mysql.connector
cone = mysql.connector.connect(
                  host = 'cloud.eant.tech',
                  database = 'pdp_base007',
                  user = 'pdp_usuario007',
                  password = 'eantpass')
cursor = cone.cursor()

#CREAMOS TABLA
tabla = 'top100cuspide'
create_query = f"""
        CREATE TABLE IF NOT EXISTS `{tabla}`
        ( `id` INT NOT NULL AUTO_INCREMENT , 
        `titulo` VARCHAR(200) NOT NULL , 
        `precio_ars` INT NOT NULL , 
        `url_libro` VARCHAR(200) NOT NULL ,
        PRIMARY KEY (`id`)) 
        ENGINE = InnoDB;
        """
cursor.execute(create_query)

---

Inserto un registro a la tabla por cada libro y por ultimo cierro la conexion

---

In [4]:
for row in resultado:
    #ITERA LA LISTA SCRAPEADA CREANDO UN ROW EN LA TABLA POR CADA LIBRO
    insert_query = f"""
        INSERT INTO top100cuspide (url_libro, titulo, precio_ars)
        VALUES ('{row[0]}', '{row[1]}', '{row[2]}');
        """
    
    cursor.execute(insert_query)
    cone.commit()

In [5]:
#CERRAMOS CONEXION   
cursor.close()
cone.close()